<a href="https://colab.research.google.com/github/cdpierse/Siamese-Network/blob/master/siamese_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Clone the entire repo.
!git clone -l -s https://github.com/cdpierse/Siamese-Network.git siamese_network
%cd siamese_network
!ls

Cloning into 'siamese_network'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6177 (delta 0), reused 1 (delta 0), pack-reused 6173
Receiving objects: 100% (6177/6177), 198.46 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Checking out files: 100% (5956/5956), done.
/content/siamese_network
data	    environment.yml  loader.py	   requirements.txt    test.py
dockerfile  __init__.py      makepairs.py  siamese_network.py


In [0]:
#!pip install -r requirements.txt

In [0]:
!python siamese_network.py